In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys, os, time, gc
import requests, shutil
from sklearn.neighbors import NearestNeighbors
import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

%matplotlib inline

# Data Information

In [ ]:
train_df = pd.read_csv('./data/triplet/train.csv')
val_df = pd.read_csv('./data/triplet/validation.csv')
test_df = pd.read_csv('./data/triplet/test.csv')

print('Train:\t\t', train_df.shape)
print('Validation:\t', val_df.shape)
print('Test:\t\t', test_df.shape)

print('\nTrain Landmarks:\t', len(train_df['landmark_id'].unique()))
print('Validation Landmarks:\t', len(val_df['landmark_id'].unique()))
print('Test Landmarks:\t\t', len(test_df['landmark_id'].unique()))

In [ ]:
train_df.head()

# Load Features and Labels

In [ ]:
# Already normalized
train_feature = np.load('./data/triplet/train_triplet_inception(1)_features.npy')
val_feature = np.load('./data/triplet/validation_triplet_inception(1)_features.npy')
test_feature = np.load('./data/triplet/test_triplet_inception(1)_features.npy')

train_df = pd.read_csv('./data/triplet/train.csv')
val_df = pd.read_csv('./data/triplet/validation.csv')
test_df = pd.read_csv('./data/triplet/test.csv')

print('Train:\t\t', train_feature.shape, train_df.shape)
print('Validation:\t', val_feature.shape, val_df.shape)
print('Test:\t\t', test_feature.shape, test_df.shape)

In [ ]:
# Helper function
def accuracy(true_label, prediction, top=1):
    """ function to calculate the prediction accuracy """
    prediction = prediction[:, :top]
    count = 0
    for i in range(len(true_label)):
        if true_label[i] in prediction[i]:
            count += 1
            
    return count / len(true_label)

# Implement KNN Model

In [ ]:
# Merge train and validation features
train_val_feature = np.concatenate((train_feature, val_feature), axis=0)
train_val_df = pd.concat((train_df, val_df), axis=0)
train_val_df = train_val_df.reset_index(drop=True)

In [ ]:
# Implement KNN model
knn = NearestNeighbors(n_neighbors=50, algorithm='auto', leaf_size=30, 
                       metric='minkowski', p=2, n_jobs=-1)
knn.fit(train_val_feature)

In [ ]:
# Search the first 50 neighbors
distance, neighbor_index = knn.kneighbors(test_feature, return_distance=True)

# Save the results
np.save('./result/knn_triplet_inception(1)_distance.npy', distance)
np.save('./result/knn_triplet_inception(1)_neighbor.npy', neighbor_index)

### Search Neighbors

In [ ]:
knn_distance = np.load('./result/knn_triplet_inception(1)_distance.npy')
knn_neighbor = np.load('./result/knn_triplet_inception(1)_neighbor.npy')

# Get the first 50 neighbors
predictions = []
for neighbors in knn_neighbor:
    predictions.append(train_val_df.loc[neighbors]['landmark_id'].values)

predictions = np.array(predictions)
np.save('./result/knn_triplet_inception(1)_test_prediction.npy', predictions)

### Compute Accuracy

In [ ]:
print('Top  1 accuracy:\t', accuracy(test_df['landmark_id'].values, predictions, top=1))
print('Top  5 accuracy:\t', accuracy(test_df['landmark_id'].values, predictions, top=5))
print('Top 10 accuracy:\t', accuracy(test_df['landmark_id'].values, predictions, top=10))
print('Top 20 accuracy:\t', accuracy(test_df['landmark_id'].values, predictions, top=20))

In [ ]:
knn_acc = []
for i in range(1, 51):
    tmp_acc = accuracy(test_df['landmark_id'].values, predictions, top=i)
    knn_acc.append(tmp_acc)

np.save('./result/knn_triplet_inception(1)_accuracy.npy', knn_acc)